# Heirarchical Model Predictive Control: HVAC Example

In [1]:
using JuMP
using Plasmo
using Ipopt
using Plots

In [2]:
# Get Neighboring Rooms
# Assume rooms are in a row

# returns a list of vectors where the index of the list is the room and the vectors are the neighboring rooms
function getNeighboringRooms(room, nrooms)
    if room == 1
        return [2,0]
    elseif room == nrooms
        return [nrooms - 1,0]
    else
        a = [room - 1,room + 1]
        return a
    end
end


getNeighboringRooms (generic function with 1 method)

In [3]:
# Thermal Dynamics into and between rooms
rho_air = 1.225 # kg/m^3
V_room = 50 # m^3

Ci = rho_air*V_room # mass of room i kg
Ti = 1 # temp of air in room in (K) 
ui = 1 # mass flowrate of air entering the room kg/s
cp = 1012 # heat capacity of air in room (J/kg*K)
Ts = 16+273 # temp of supply air (K)
Toa = 293 # temp of outside air (K)
Rij = 4 # thermal resistance between rooms i and j (K/kW)
Rioa = 57 # thermal resistance between room and outside (K/kW)
Pid = 1# disturbance term from weather forecast

#Ci*(dTi)=dt*(ui*cp*(Ts−Ti)+∑((Tj−Ti)/Rij+(Toa−Ti)/Rioa)+Pid)

1

In [41]:
# function HVAC()
# find constraints, objective variables, setpoints,

# Constraints:
Ti = [293, 297] # K
Td = 296 # desired temp of room
ui = 1 # must be within [.005, 5] # kg/s

# Tuning Parameters
c1 = 1
c2 = 2

# Objective variables
# Ti must be within 20 and 24 

# function inputs
n_rooms = 2
N_U = 2 # upper level nodes
N_L = 8 # lower level nodes

# Sampling periods (hours)
dt_U = 1 
dt_L = .25

# Time Horizons
horiz_U = 24 # hours
horiz_L = 2 # hours

HVAC = Model(Ipopt.Optimizer)
graph = OptiGraph()

@optinode(graph, Unodes[1:N_U]) # make N_U upper layer nodes
@optinode(graph, Lnodes[1:N_L]) # make N_L lower layer nodes

# Upper Level Initialization
for (i, Unode) in enumerate(Unodes)
    
    # array belonging to upper layer containing temps of each room in the node
    @variable(Unode, T_hat_U_array[1:n_rooms, 1:(floor(Int, (N_L/N_U)))], start = 300)
        # row index: room number
        # column index: time
    
    # array belonging to upper layer containing airflowrate into each room
    @variable(Unode, u_hat_U_array[1:n_rooms, 1:(floor(Int, (N_L/N_U)))], start = 1)
        # row index: room number
        # column index: time
    @variable(Unode, T_0_array[1:n_rooms, 1:(floor(Int, (N_L/N_U)))], start = 300) # 2x4 array
        # row index: room number
        # column index: time
    @objective(Unode, Min, sum(.01*(T_hat_U_array .- Td).^2 .+ 100*(u_hat_U_array).^2))
end

# Lower Level Initailization
for (i, Lnode) in enumerate(Lnodes)
    @variable(Lnode, 293 <= T_hat_L_array[1:n_rooms] <= 297, start = 295) # temp of each room
    @variable(Lnode, .005 <= u_hat_L_array[1:n_rooms] <= 5, start = 1) # airflowrate of each room
    
end

# Objective functions


# Link Constraints between Unodes
for Unode_Index in 1:(length(Unodes) - 1)
    for (room_num, T_room_num) in enumerate(Unodes[Unode_Index][:T_hat_U_array][:,1]) # only use temps of rooms at earliest time
        if room_num == 1 # if in the first room
            Tj = Unodes[Unode_Index][:T_hat_U_array][room_num + 1, 1] # establish neighboring room
            @linkconstraint(graph, Unodes[Unode_Index + 1][:T_hat_U_array][room_num] == 
                dt_U/Ci*(ui*cp*(Ts − T_room_num) + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps
            
        elseif room_num == n_rooms # if in the last last room
            Tj = Unodes[Unode_Index][:T_hat_U_array][room_num - 1, 1]
            @linkconstraint(graph, (Unodes[Unode_Index + 1][:T_hat_U_array][room_num]) == 
                dt_U/Ci*(ui*cp*(Ts − T_room_num) + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps
            
        else # if in a room surrounded by 2 rooms
            Tj1 = Unodes[Unode_Index][:T_hat_U_array][room_num - 1, 1]
            Tj2 = Unodes[Unode_Index][:T_hat_U_array][room_num + 1, 1]
            @linkconstraint(graph, (Unodes[Unode_Index + 1][:T_hat_U_array][room_num]) == 
                dt_U/Ci*(ui*cp*(Ts − T_room_num) + ((Tj1 − T_room_num)/Rij + (Tj2 − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps
        end
    end
    @objective(Unodes[Unode_Index], Min, 
        Unodes[Unode_Index][:T_hat_U_array][1] + 
        Unodes[Unode_Index][:u_hat_U_array][1] + 
        Unodes[Unode_Index][:T_hat_U_array][2] +
        Unodes[Unode_Index][:u_hat_U_array][2])
end



# Link Constraints between Unodes and Lnodes
for i in range(1,N_U) # loop through each U node
    
    for j in range(1,floor(Int, N_L/2)) # loop through 4 Lnodes at a time
        Lnode_Index = floor(Int, j + N_L*(i - 1)/2)
        
        #create link constraint between them
        for room_index in range(1, n_rooms) # add a link constraint for each room
            
            # represent the room of the specific Lnode that the Unode i will link to 
            T_hat_l_room = Lnodes[Lnode_Index][:T_hat_L_array][room_index]
            
            # represent the upper layer temp solution for the lower layer
            T_0 = Unodes[i][:T_0_array][room_index, j]
            
            @linkconstraint(graph, (T_hat_l_room) == 1/Ci*(ui*cp*(Ts − T_hat_l_room) 
                + ((T_0 − T_hat_l_room)/Rij + (Toa − T_hat_l_room)/Rioa) + Pid))
        end
    end
end


# Link Constraints Between Lnodes
for Lnode_Index in 1:(length(Lnodes) - 1) # iterate through all Lnodes except for last
    for (room_num, T_room_num) in enumerate(Lnodes[Lnode_Index][:T_hat_L_array]) # iterate through each room
        
        if room_num == 1 # if in the first room
            Tj = Lnodes[Lnode_Index][:T_hat_L_array][room_num + 1] # establish neighboring room
            @linkconstraint(graph, Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num] == 
                dt_L/Ci*(ui*cp*(Ts − T_room_num) + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps
            
        elseif room_num == n_rooms # if in the last last room
            Tj = Lnodes[Lnode_Index][:T_hat_L_array][room_num - 1]
            @linkconstraint(graph, (Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num]) == 
                dt_L/Ci*(ui*cp*(Ts − T_room_num) + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps
            
        else # if in a room surrounded by 2 rooms
            Tj1 = Lnodes[Lnode_Index][:T_hat_L_array][room_num - 1]
            Tj2 = Lnodes[Lnode_Index][:T_hat_L_array][room_num + 1]
            @linkconstraint(graph, (Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num]) == 
                dt_L/Ci*(ui*cp*(Ts − T_room_num) + ((Tj1 − T_room_num)/Rij + (Tj2 − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps
            
        end
    end
        
    @objective(Lnodes[Lnode_Index], Min, 
        Lnodes[Lnode_Index][:T_hat_L_array][1] + 
        Lnodes[Lnode_Index][:u_hat_L_array][1] + 
        Lnodes[Lnode_Index][:T_hat_L_array][2] +
        Lnodes[Lnode_Index][:u_hat_L_array][2])
end





Lnodes[3][:T_hat_L_array[2]]

VariableRef[Unodes[1][:T_hat_U_array[1,1]], Unodes[1][:T_hat_U_array[2,1]]]

1